# Model Calibration


In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using CSV
using DataFrames
using DiffFusion
using PlotlyJS

In [ ]:
include("utils.jl")

## Reference Data

### Volatility Data

In [ ]:
file_name = "data/standard_deviation_30days.csv"
std_table = DataFrame(CSV.File(file_name))
std_table = stack(std_table, 3:size(std_table)[2])
std_table[!, "VOLATILITY"] = std_table[!, "value"] / sqrt(30.0/365.0);
std_table[!, "YEARS"] = std_table[!, "MONTHS"] / 12;

### Correlation Data

In [ ]:
file_name = "data/correlations_30days.csv"
corr_table = DataFrame(CSV.File(file_name))
corr_table = stack(corr_table, 5:size(corr_table)[2])
corr_table[!, "YEARS1"] = Int.(corr_table[!, "MONTHS1"] / 12)
corr_table[!, "YEARS2"] = Int.(corr_table[!, "MONTHS2"] / 12);

## Model Setup

In [ ]:
model_params = Dict([
    (("delta_1", ""), 1.0),
    (("delta_2", ""), 10.0),
    (("delta_3", ""), 20.0),
    #
    (("EUR_chi_1", ""), 0.01),
    (("EUR_chi_2", ""), 0.50),
    (("EUR_chi_3", ""), 0.80),
    #
    (("USD_chi_1", ""), 0.01),
    (("USD_chi_2", ""), 0.30),
    (("USD_chi_3", ""), 0.60),
    #
    (("GBP_chi_1", ""), 0.01),
    (("GBP_chi_2", ""), 0.20),
    (("GBP_chi_3", ""), 0.40),
    #
    (("EUR_f_1", ""), 0.0060),
    (("EUR_f_2", ""), 0.0068),
    (("EUR_f_3", ""), 0.0071),
    #
    (("USD_f_1", ""), 0.0070),
    (("USD_f_2", ""), 0.0085),
    (("USD_f_3", ""), 0.0080),
    #
    (("GBP_f_1", ""), 0.0078),
    (("GBP_f_2", ""), 0.0086),
    (("GBP_f_3", ""), 0.0077),
    #
    (("USD-EUR_x", ""), 0.090),
    (("GBP-EUR_x", ""), 0.075),
    #
    (("EUR_f_1", "EUR_f_2"), 0.55),
    (("EUR_f_2", "EUR_f_3"), 0.95),
    (("EUR_f_1", "EUR_f_3"), 0.45),
    #
    (("USD_f_1", "USD_f_2"), 0.55),
    (("USD_f_2", "USD_f_3"), 0.95),
    (("USD_f_1", "USD_f_3"), 0.45),
    #
    (("GBP_f_1", "GBP_f_2"), 0.60),
    (("GBP_f_2", "GBP_f_3"), 0.95),
    (("GBP_f_1", "GBP_f_3"), 0.50),
    #
    (("EUR_f_1", "USD-EUR_x"), 0.25),
    (("EUR_f_2", "USD-EUR_x"), 0.10),
    (("EUR_f_3", "USD-EUR_x"), 0.08),
    #
    (("USD_f_1", "USD-EUR_x"), -0.15),
    (("USD_f_2", "USD-EUR_x"), -0.10),
    (("USD_f_3", "USD-EUR_x"), -0.08),
    #
    (("GBP_f_1", "USD-EUR_x"), 0.05),
    (("GBP_f_2", "USD-EUR_x"), -0.05),
    (("GBP_f_3", "USD-EUR_x"), -0.07),
    #
    (("EUR_f_1", "GBP-EUR_x"), 0.07),
    (("EUR_f_2", "GBP-EUR_x"), 0.03),
    (("EUR_f_3", "GBP-EUR_x"), 0.02),
    #
    (("USD_f_1", "GBP-EUR_x"), -0.18),
    (("USD_f_2", "GBP-EUR_x"), -0.22),
    (("USD_f_3", "GBP-EUR_x"), -0.20),
    #
    (("GBP_f_1", "GBP-EUR_x"), -0.25),
    (("GBP_f_2", "GBP-EUR_x"), -0.17),
    (("GBP_f_3", "GBP-EUR_x"), -0.13),
    #
    (("USD-EUR_x", "GBP-EUR_x"), 0.35),
    #
    (("EUR_f_1", "USD_f_1"), 0.55),
    (("EUR_f_1", "USD_f_2"), 0.45),
    (("EUR_f_1", "USD_f_3"), 0.35),
    #
    (("EUR_f_2", "USD_f_1"), 0.40),
    (("EUR_f_2", "USD_f_2"), 0.75),
    (("EUR_f_2", "USD_f_3"), 0.75),
    #
    (("EUR_f_3", "USD_f_1"), 0.35),
    (("EUR_f_3", "USD_f_2"), 0.70),
    (("EUR_f_3", "USD_f_3"), 0.75),
    #
    (("EUR_f_1", "GBP_f_1"), 0.67),
    (("EUR_f_1", "GBP_f_2"), 0.47),
    (("EUR_f_1", "GBP_f_3"), 0.37),
    #
    (("EUR_f_2", "GBP_f_1"), 0.45),
    (("EUR_f_2", "GBP_f_2"), 0.80),
    (("EUR_f_2", "GBP_f_3"), 0.75),
    #
    (("EUR_f_3", "GBP_f_1"), 0.37),
    (("EUR_f_3", "GBP_f_2"), 0.75),
    (("EUR_f_3", "GBP_f_3"), 0.75),
    #
    (("USD_f_1", "GBP_f_1"), 0.55),
    (("USD_f_1", "GBP_f_2"), 0.40),
    (("USD_f_1", "GBP_f_3"), 0.35),
    #
    (("USD_f_2", "GBP_f_1"), 0.45),
    (("USD_f_2", "GBP_f_2"), 0.77),
    (("USD_f_2", "GBP_f_3"), 0.72),
    #
    (("USD_f_3", "GBP_f_1"), 0.32),
    (("USD_f_3", "GBP_f_2"), 0.77),
    (("USD_f_3", "GBP_f_3"), 0.75),    
]);

## Model Analysis

In [ ]:
update_hybrid_plots!(
    [
    ],
    model_params, std_table, corr_table,
    plot_vols = true,
    plot_rates_corrs = true,
    plot_fx_corrs = true,
    plot_fx_rates_corrs = true,
    plot_rates_rates_corrs = true,
)